<a href="https://colab.research.google.com/github/biwnunticha/MT-for-songs-EN-TH/blob/main/Fairseq_MT_(Songs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Libraries and Dataset


In [ ]:
%%capture
# W and B -- For Logging
! pip install wandb

# Sacremoses -- For Tokenizing
! pip install sacremoses

# fairseq -- For training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

In [ ]:
!pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 20.5 MB/s 


In [ ]:
!pip install attacut

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 29.9 MB/s 
     |████████████████████████████████| 473 kB 90.0 MB/s 
     |████████████████████████████████| 87 kB 8.1 MB/s 
     |████████████████████████████████| 1.0 MB 86.4 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=ac7ea797fb4d2f3f536e130bad87f9f5a29c016d99a291122f0084e4f6a0e0af
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=3e5b71a4af7c4de00860ae8d3aae005606a224d30213d0c94be29dd5095f5d78
  Stored in directory: /root/.cache/pip/wheels/1f/10/06/2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
Successfully built docopt fire


In [ ]:
import sys
import pdb
import pprint
import logging
import os
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import numpy as np
import tqdm.auto as tqdm
from pathlib import Path
from argparse import Namespace
#from fairseq import utils

import matplotlib.pyplot as plt

In [ ]:
!pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 33.8 MB/s 


In [ ]:
!pip install git+https://github.com/MCFreddie777/language-check.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MCFreddie777/language-check.git to /tmp/pip-req-build-em0q3w6v
  Running command git clone -q https://github.com/MCFreddie777/language-check.git /tmp/pip-req-build-em0q3w6v
  Created wheel for language-check: filename=language_check-1.1-py3-none-any.whl size=90190998 sha256=555d5879bed20b5dfc8b15c320c9e914ef3be6f3cbe0e26a23821d63fca83231
  Stored in directory: /tmp/pip-ephem-wheel-cache-ehf41tz1/wheels/c6/a8/a2/992f0138466a1425526af353678322a252aa8cbd0a6d93e2e7
Successfully built language-check


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 25.9 MB/s 
     |████████████████████████████████| 110 kB 98.6 MB/s 


In [ ]:
import pandas as pd
import pythainlp
import string
import contractions
import re

In [ ]:
from pythainlp.tokenize import word_tokenize

In [ ]:
!gdown --id 1svxRO87Z2oGJ2k29AjeQ_iTf4luxXr0I
!gdown --id 13iwodohZ4L18k5kIWh1mBs2r7VVvjqVf
!gdown --id 14c4reG13pggu605Ouh4gQhfHM6ZM8HZk

In [ ]:
train = pd.read_csv('train.csv')
dev = pd.read_csv('dev.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.columns

In [ ]:
train = train.rename(columns={'input_text':'lyrics', 'target_text':'tokenized'})
dev = dev.rename(columns={'input_text':'lyrics', 'target_text':'tokenized'})

In [ ]:
with open('train.th', mode='w') as train_th:
  for line in train['tokenized']:
    line = str(line)
    train_th.write(line.strip())
    train_th.write('\n')

In [ ]:
with open('train.en', mode='w') as train_en:
  for line in train['lyrics']:
    line = str(line)
    train_en.write(line.strip())
    train_en.write('\n')

In [ ]:
with open('train.th', mode='w') as train_th:
  for line in train['tokenized']:
    line = str(line)
    train_th.write(line.strip())
    train_th.write('\n')

In [ ]:
with open('dev.en', mode='w') as dev_en:
  for line in dev['lyrics']:
    line = str(line)
    dev_en.write(line.strip())
    dev_en.write('\n')

In [ ]:
with open('dev.th', mode='w') as dev_th:
  for line in dev['tokenized']:
    line = str(line)
    dev_th.write(line.strip())
    dev_th.write('\n')

In [ ]:
with open('test.en', mode='w') as test_en:
  for line in test['lyrics']:
    line = str(line)
    test_en.write(line.strip())
    test_en.write('\n')

In [ ]:
with open('test.th', mode='w') as test_th:
  for line in test['tokenized']:
    line = str(line)
    test_th.write(line.strip())
    test_th.write('\n')

## Pre-process and Binarize to build Vocabularies

In [ ]:
! fairseq-preprocess --source-lang en --target-lang th \
  --trainpref /content/train\
  --validpref  /content/dev\
  --testpref /content/test\
  --destdir data-bin/lyrics2022\
  --thresholdsrc 2 \
  --thresholdtgt 2

# Train the model

In [ ]:
! fairseq-train  'data-bin/lyrics2022'\
  --arch transformer \
  --dropout 0.1 \
  --attention-dropout 0.1 \
  --activation-dropout 0.1 \
  --encoder-embed-dim 256 \
  --encoder-ffn-embed-dim 512 \
  --encoder-layers 3 \
  --encoder-attention-heads 8 \
  --encoder-learned-pos \
  --decoder-embed-dim 256 \
  --decoder-ffn-embed-dim 512 \
  --decoder-layers 3 \
  --decoder-attention-heads 8 \
  --decoder-learned-pos \
  --max-epoch 10 \
  --optimizer adam \
  --lr 5e-4 \
  --batch-size 128 \
  --seed 1 \
  # --wandb-project "Lyrics en to th Translation"

# Evaluate the model

In [ ]:
from sacrebleu import tokenizers

In [ ]:
# ckpt_best, beam=5
! fairseq-generate data-bin/lyrics2022 \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    # --wandb-project "Multi 30K En to De Translation"

In [ ]:
# ckpt_best, beam=10
! fairseq-generate data-bin/lyrics2022 \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 10 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    # --wandb-project "Multi 30K En to De Translation"

# Generate Average Checkpoint (Ensemble)

In [ ]:
!python fairseq/scripts/average_checkpoints.py --inputs checkpoints --num-epoch-checkpoints 5 --output checkpoints/averaged.pt

In [ ]:
# ckpt_avg, beam=5
! fairseq-generate data-bin/lyrics2022 \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    # --wandb-project "Multi 30K En to De Translation"

In [ ]:
# ckpt_avg, beam=10
! fairseq-generate data-bin/lyrics2022 \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 10 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    # --wandb-project "Multi 30K En to De Translation"

# Export the model

In [ ]:
! mkdir -p trained_model
! cp data-bin/lyrics2022/dict.th.txt trained_model/dict.th.txt
! cp data-bin/lyrics2022/dict.en.txt trained_model/dict.en.txt
! cp checkpoints/averaged.pt trained_model/model.pt

In [ ]:
! fairseq-generate data-bin/lyrics2022 \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    --valid-subset valid

In [ ]:
! fairseq-interactive \
  --path trained_model/model.pt \
  --source-lang en --target-lang th \
  --tokenizer moses \
  --task translation --cpu \
  --beam 5 \
  trained_model/

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

In [ ]:
from fairseq.models.transformer import TransformerModel
model = TransformerModel.from_pretrained(
  '/content/trained_model',
  checkpoint_file='model.pt',
  source_lang='en',
  target_lang='th'
  #data_name_or_path='test.csv',
  #bpe='fastbpe'
  #bpe_codes=<bpe_file>
)

In [ ]:
pred_list = []
gold_list = []
for index, lyrics in enumerate(test['lyrics']):
  pred = model.translate(lyrics).replace(' ','')
  pred_list.append(pred)
  pred = pred.replace(' ', '').replace('▁', ' ').strip()
  gold = test.iloc[index].tokenized.replace(' ','')
  gold_list.append(gold)
  input_sentence = lyrics
  print('\ninput_sentence:', input_sentence)
  print('gold:', gold)
  print('translation:', pred)



In [ ]:
test_list = test['lyrics'].tolist()

In [ ]:
pred_df = pd.DataFrame({'input_text': test_list, 'predict': [x.replace(' ','').replace('<unk>','') for x in pred_list], 'gold': [x.replace(' ','').replace('<unk>','') for x in gold_list]})

In [ ]:
pred_df

In [ ]:
pred_df.to_csv('fairseq_prediction.csv')

In [ ]:
pred_list[58]

In [ ]:
gold_list[58]

In [ ]:
chrf = evaluate.load("chrf")
results = chrf.compute(predictions=pred_list,references=gold_list,word_order=2,lowercase=True)
print(results)